# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse       Shipments  \
0  Jul  6 2022 11:21AM  241201        20  ACI-8182208-BO   
1  Jul  6 2022 11:21AM  241201        20         8234590   
2  Jul  6 2022 11:21AM  241201        20         8234591   
3  Jul  6 2022 11:21AM  241201        20         8234592   
4  Jul  6 2022 11:21AM  241201        20         8234628   
5  Jul  6 2022 11:21AM  241201        20         8234629   
6  Jul  6 2022 11:21AM  241201        20         8234630   
7  Jul  6 2022 11:21AM  241201        20         8235851   
8  Jul  6 2022 11:21AM  241201        20         8235852   
9  Jul  6 2022 11:21AM  241201        20         8236423   

                   Customer ShipmentStatus  
0  ACI Healthcare USA, Inc.       Released  
1  ACI Healthcare USA, Inc.       Released  
2  ACI Healthcare USA, Inc.       Released  
3  ACI Healthcare USA, Inc.       Released  
4  ACI Healthcare USA, Inc.       Released  
5  ACI Healthcare USA, Inc.       Released  
6  ACI Healthcare USA, Inc.       Released  
7  ACI Healthcare USA, Inc.       Released  
8  ACI Healthcare USA, Inc.       Released  
9  ACI Healthcare USA, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
64  241196       Released          4
65  241197       Released          5
66  241198       Released          1
67  241200       Released          1
68  241201       Released         13

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
241196                NaN        NaN       4.0
241197                NaN        NaN       5.0
241198                NaN        NaN       1.0
241200                NaN        NaN       1.0
241201                NaN        NaN      13.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
241087                0.0       18.0      12.0
241095                0.0        0.0       8.0
241100                0.0        0.0       1.0
241101                0.0        2.0       2.0
241103                0.0        4.0      11.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
241087                  0         18        12
241095                  0          0         8
241100                  0          0         1
241101                  0          2         2
241103                  0          4        11

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               241087          0         18        12
1               241095          0          0         8
2               241100          0          0         1
3               241101          0          2         2
4               241103          0          4        11

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               241087                  18       12
1               241095                            8
2               241100                            1
3               241101                   2        2
4               241103                   4       11

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse  \
0    Jul  6 2022 11:21AM  241201        20   
13   Jul  6 2022 11:13AM  241200        10   
14   Jul  6 2022 10:53AM  241197        15   
19   Jul  6 2022 10:52AM  241196        15   
23   Jul  6 2022 10:52AM  241198        19   
24   Jul  6 2022 10:42AM  241193        15   
44   Jul  6 2022 10:40AM  241192        10   
61   Jul  6 2022 10:38AM  241190        15   
98   Jul  6 2022 10:30AM  241191        10   
100  Jul  6 2022 10:30AM  241189        10   

                                  Customer  
0                 ACI Healthcare USA, Inc.  
13                        Methapharm, Inc.  
14   Carwin Pharmaceutical Associates, LLC  
19              Laser Pharmaceuticals, LLC  
23          Avet Pharmaceuticals Labs, Inc  
24         Brookfield Pharmaceuticals, LLC  
44                        Eywa Pharma Inc.  
61                   Alliance Pharma, Inc.  
98                       ISDIN Corporation  
100                      ISDIN Corporation

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse  \
0  Jul  6 2022 11:21AM  241201        20   
1  Jul  6 2022 11:13AM  241200        10   
2  Jul  6 2022 10:53AM  241197        15   
3  Jul  6 2022 10:52AM  241196        15   
4  Jul  6 2022 10:52AM  241198        19   
5  Jul  6 2022 10:42AM  241193        15   
6  Jul  6 2022 10:40AM  241192        10   
7  Jul  6 2022 10:38AM  241190        15   
8  Jul  6 2022 10:30AM  241191        10   
9  Jul  6 2022 10:30AM  241189        10   

                                Customer Completed Executing Released  
0               ACI Healthcare USA, Inc.                           13  
1                       Methapharm, Inc.                            1  
2  Carwin Pharmaceutical Associates, LLC                            5  
3             Laser Pharmaceuticals, LLC                            4  
4         Avet Pharmaceuticals Labs, Inc                            1  
5        Brookfield Pharmaceuticals, LLC                  13        7  
6                       Eywa Pharma Inc.                           17  
7                  Alliance Pharma, Inc.                  10       27  
8                      ISDIN Corporation                            2  
9                      ISDIN Corporation                           46

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse  \
0  Jul  6 2022 11:21AM  241201        20   
1  Jul  6 2022 11:13AM  241200        10   
2  Jul  6 2022 10:53AM  241197        15   
3  Jul  6 2022 10:52AM  241196        15   
4  Jul  6 2022 10:52AM  241198        19   
5  Jul  6 2022 10:42AM  241193        15   
6  Jul  6 2022 10:40AM  241192        10   
7  Jul  6 2022 10:38AM  241190        15   
8  Jul  6 2022 10:30AM  241191        10   
9  Jul  6 2022 10:30AM  241189        10   

                                Customer Released Executing Completed  
0               ACI Healthcare USA, Inc.       13                      
1                       Methapharm, Inc.        1                      
2  Carwin Pharmaceutical Associates, LLC        5                      
3             Laser Pharmaceuticals, LLC        4                      
4         Avet Pharmaceuticals Labs, Inc        1                      
5        Brookfield Pharmaceuticals, LLC        7        13            
6                       Eywa Pharma Inc.       17                      
7                  Alliance Pharma, Inc.       27        10            
8                      ISDIN Corporation        2                      
9                      ISDIN Corporation       46

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse  \
0  Jul  6 2022 11:21AM  241201        20   
1  Jul  6 2022 11:13AM  241200        10   
2  Jul  6 2022 10:53AM  241197        15   
3  Jul  6 2022 10:52AM  241196        15   
4  Jul  6 2022 10:52AM  241198        19   

                                Customer Released Executing Completed  
0               ACI Healthcare USA, Inc.       13                      
1                       Methapharm, Inc.        1                      
2  Carwin Pharmaceutical Associates, LLC        5                      
3             Laser Pharmaceuticals, LLC        4                      
4         Avet Pharmaceuticals Labs, Inc        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse  \
0  Jul  6 2022 11:21AM  241201        20   
1  Jul  6 2022 11:13AM  241200        10   
2  Jul  6 2022 10:53AM  241197        15   
3  Jul  6 2022 10:52AM  241196        15   
4  Jul  6 2022 10:52AM  241198        19   

                                Customer  Released  Executing  Completed  
0               ACI Healthcare USA, Inc.      13.0        NaN        NaN  
1                       Methapharm, Inc.       1.0        NaN        NaN  
2  Carwin Pharmaceutical Associates, LLC       5.0        NaN        NaN  
3             Laser Pharmaceuticals, LLC       4.0        NaN        NaN  
4         Avet Pharmaceuticals Labs, Inc       1.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse  \
0  Jul  6 2022 11:21AM  241201        20   
1  Jul  6 2022 11:13AM  241200        10   
2  Jul  6 2022 10:53AM  241197        15   
3  Jul  6 2022 10:52AM  241196        15   
4  Jul  6 2022 10:52AM  241198        19   

                                Customer  Released  Executing  Completed  
0               ACI Healthcare USA, Inc.      13.0        0.0        0.0  
1                       Methapharm, Inc.       1.0        0.0        0.0  
2  Carwin Pharmaceutical Associates, LLC       5.0        0.0        0.0  
3             Laser Pharmaceuticals, LLC       4.0        0.0        0.0  
4         Avet Pharmaceuticals Labs, Inc       1.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         7475922     451.0        1.0        3.0
12          964560      24.0        5.0        0.0
15          964776      43.0       23.0        0.0
16         1687865       9.0        7.0        0.0
19          964587      58.0       34.0       13.0
20         1205678     105.0       28.0        0.0
21          241183       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  7475922     451.0        1.0        3.0
1        12   964560      24.0        5.0        0.0
2        15   964776      43.0       23.0        0.0
3        16  1687865       9.0        7.0        0.0
4        19   964587      58.0       34.0       13.0
5        20  1205678     105.0       28.0        0.0
6        21   241183       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     451.0        1.0        3.0
1        12      24.0        5.0        0.0
2        15      43.0       23.0        0.0
3        16       9.0        7.0        0.0
4        19      58.0       34.0       13.0
5        20     105.0       28.0        0.0
6        21       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  451.0
1        12  Released   24.0
2        15  Released   43.0
3        16  Released    9.0
4        19  Released   58.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    12    15   16    19     20   21
Status                                             
Completed    3.0   0.0   0.0  0.0  13.0    0.0  0.0
Executing    1.0   5.0  23.0  7.0  34.0   28.0  0.0
Released   451.0  24.0  43.0  9.0  58.0  105.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    12    15   16    19     20   21
0          Completed    3.0   0.0   0.0  0.0  13.0    0.0  0.0
1          Executing    1.0   5.0  23.0  7.0  34.0   28.0  0.0
2           Released  451.0  24.0  43.0  9.0  58.0  105.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    12    15   16    19     20   21
0  Completed    3.0   0.0   0.0  0.0  13.0    0.0  0.0
1  Executing    1.0   5.0  23.0  7.0  34.0   28.0  0.0
2   Released  451.0  24.0  43.0  9.0  58.0  105.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()